In [1]:
# Imports and load data
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load dataset
df = pd.read_csv(r"..\data\raw\marketing_campaign.csv", encoding='ISO-8859-1', sep="\t") 

print("Initial shape:", df.shape)
print(df.head(3))
print(df.info())

Initial shape: (2240, 29)
     ID  Year_Birth   Education Marital_Status   Income  Kidhome  Teenhome  \
0  5524        1957  Graduation         Single  58138.0        0         0   
1  2174        1954  Graduation         Single  46344.0        1         1   
2  4141        1965  Graduation       Together  71613.0        0         0   

  Dt_Customer  Recency  MntWines  ...  NumWebVisitsMonth  AcceptedCmp3  \
0  04-09-2012       58       635  ...                  7             0   
1  08-03-2014       38        11  ...                  5             0   
2  21-08-2013       26       426  ...                  4             0   

   AcceptedCmp4  AcceptedCmp5  AcceptedCmp1  AcceptedCmp2  Complain  \
0             0             0             0             0         0   
1             0             0             0             0         0   
2             0             0             0             0         0   

   Z_CostContact  Z_Revenue  Response  
0              3         11         1  

In [2]:
# -------------------------------
#  Data Cleaning
# -------------------------------

# Drop constant columns (like Z_Revenue or Z_CostContact)
df = df.drop(columns=[col for col in df.columns if df[col].nunique() == 1])  

print("After dropping duplicates and constant columns:", df.shape)

After dropping duplicates and constant columns: (2240, 27)


In [3]:
# Impute missing income with median
df['Income'] = df['Income'].fillna(df['Income'].median())  

# Check missing values
df.isnull().sum()

ID                     0
Year_Birth             0
Education              0
Marital_Status         0
Income                 0
Kidhome                0
Teenhome               0
Dt_Customer            0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
AcceptedCmp3           0
AcceptedCmp4           0
AcceptedCmp5           0
AcceptedCmp1           0
AcceptedCmp2           0
Complain               0
Response               0
dtype: int64

In [4]:
# Convert to datetime
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%d-%m-%Y')  

# Calculate age
current_year = pd.Timestamp.today().year
df['Age'] = current_year - df['Year_Birth']

# Quick check
df[['Dt_Customer','Year_Birth','Age']].head(5)

,Dt_Customer,Year_Birth,Age
0,2012-09-04,1957,69
1,2014-03-08,1954,72
2,2013-08-21,1965,61
3,2014-02-10,1984,42
4,2014-01-19,1981,45


In [5]:
# Remove outliers

# Age outliers using IQR
Q1_age = df['Age'].quantile(0.25)
Q3_age = df['Age'].quantile(0.75)
IQR_age = Q3_age - Q1_age
lower_age = Q1_age - 1.5 * IQR_age
upper_age = Q3_age + 1.5 * IQR_age
df = df[(df['Age'] >= lower_age) & (df['Age'] <= upper_age)]

# Income outliers using IQR
Q1_income = df['Income'].quantile(0.25)
Q3_income = df['Income'].quantile(0.75)
IQR_income = Q3_income - Q1_income
lower_income = Q1_income - 1.5 * IQR_income
upper_income = Q3_income + 1.5 * IQR_income
df = df[(df['Income'] >= lower_income) & (df['Income'] <= upper_income)]

print("After outlier removal:", df.shape)

After outlier removal: (2229, 28)


In [6]:
# Standardize categorical text
df['Education'] = df['Education'].str.strip().str.lower()  # Remove extra spaces + standardize casing
df['Marital_Status'] = df['Marital_Status'].str.strip().str.lower()

print("Unique Education:", df['Education'].unique())
print("Unique Marital_Status:", df['Marital_Status'].unique())

Unique Education: <StringArray>
['graduation', 'phd', 'master', 'basic', '2n cycle']
Length: 5, dtype: str
Unique Marital_Status: <StringArray>
['single', 'together', 'married', 'divorced', 'widow', 'alone', 'absurd',
 'yolo']
Length: 8, dtype: str


In [7]:
# -------------------------------
#  Feature Engineering
# -------------------------------

df['Customer_Tenure_Days'] = (pd.Timestamp.today() - df['Dt_Customer']).dt.days  # Tenure in days

# Spending columns
spend_cols = ['MntWines','MntFruits','MntMeatProducts','MntFishProducts','MntSweetProducts','MntGoldProds']

# Log-transform skewed spending
df[spend_cols] = df[spend_cols].apply(lambda x: np.log1p(x))  # Reduce skew

# Total spending across products
df['Total_Spending'] = df[spend_cols].sum(axis=1)

# Total purchases across channels
purchase_cols = ['NumDealsPurchases','NumWebPurchases','NumCatalogPurchases','NumStorePurchases']
df['Total_Purchases'] = df[purchase_cols].sum(axis=1)

In [8]:
# -------------------------------
#  New Features
# -------------------------------

# Total children at home
df['Children_at_home'] = df['Kidhome'] + df['Teenhome']

# Consolidate Marital_Status into Living_With
df["Living_With"] = df["Marital_Status"].replace({
    "married":"Partner", "together":"Partner",
    "absurd":"Alone", "widow":"Alone", "yolo":"Alone",
    "divorced":"Alone", "single":"Alone", "alone":"Alone"
})

# Consolidate Education categories
df['Education'] = df['Education'].replace({
    'basic':'Undergraduate',
    '2n cycle':'Undergraduate',
    'graduation':'Graduate',
    'master':'Postgraduate',
    'phd':'PhD',
    })

# Map Living_With to numeric for family size calculation
df['Living_With_Num'] = df['Living_With'].map({"Alone":1,"Partner":2})
df['Living_With_Num'] = df['Living_With_Num'].fillna(1)  # Any unexpected ==> Alone

# Total family members
df['Family_Size'] = df['Living_With_Num'] + df['Children_at_home']

In [9]:
# -------------------------------
#  DROP NOT NEEDED / REDUNDANT / INTERMEDIATE COLUMNS
# -------------------------------

df = df.drop(columns=[
    'Kidhome',            # Replaced by Children_at_home
    'Teenhome',           # Replaced by Children_at_home
    'Marital_Status',     # Replaced by Living_With
    'Living_With_Num',    # Temporary column for Family_Size
    'Dt_Customer',        # Replaced by Customer_Tenure_Days
     'ID',                # Not needed
     'Year_Birth'         # Replaced by Age
])

In [10]:
# -------------------------------
#  Check Preprocessed Data
# -------------------------------

print(df.head())
print(df.info())

  Education   Income  Recency  MntWines  MntFruits  MntMeatProducts  \
0  Graduate  58138.0       58  6.455199   4.488636         6.304449   
1  Graduate  46344.0       38  2.484907   0.693147         1.945910   
2  Graduate  71613.0       26  6.056784   3.912023         4.852030   
3  Graduate  26646.0       26  2.484907   1.609438         3.044522   
4       PhD  58293.0       94  5.159055   3.784190         4.779123   

   MntFishProducts  MntSweetProducts  MntGoldProds  NumDealsPurchases  ...  \
0         5.153292          4.488636      4.488636                  3  ...   
1         1.098612          0.693147      1.945910                  2  ...   
2         4.718499          3.091042      3.761200                  1  ...   
3         2.397895          1.386294      1.791759                  2  ...   
4         3.850148          3.332205      2.772589                  5  ...   

   AcceptedCmp2  Complain  Response  Age  Customer_Tenure_Days  \
0             0         0         1   

In [11]:
# -------------------------------
#  Save Preprocessed Data
# -------------------------------

df.to_csv(r"..\data\preprocessed\marketing_campaign_preprocessed.csv", index=False)

In [12]:
# -------------------------------
#  Output shapes
# -------------------------------
print("Cleaned dataframe shape:", df.shape)
print("Files saved to '..\\data\\preprocessed\\'")

Cleaned dataframe shape: (2229, 28)
Files saved to '..\data\preprocessed\'
